In [ ]:
import time
import datetime
from functools import partial
from pathlib import Path
import subprocess

from tqdm.notebook import tqdm, trange
from rich.console import Console
import numpy as np
import ipywidgets as widgets

from mmwave.fmc4030 import FMC4030, Braket
from mmwave.fmc4030.bracket import cal_running_time, cal_acc_time_length
from mmwave.mmwave import MMWaveCmd
from mmwave.util import turn_toml
from mmwave.config import short_range_cfg as mmw_cfg

print = Console().print
debug = True


def cal_last(use_num, step):
    return step - use_num % step


def cal_all_time(axis_info, step_num):
    all_time = 0
    for axi_info in axis_info:
        axi_time = cal_running_time(*axi_info) * step_num
        all_time += axi_time
    all_time += step_num * 0.1
    return all_time


inslider_style = {"description_width": "initial", "width": "50%"}
inslider_layout = widgets.Layout(width="50%")

IntSlider = partial(widgets.IntSlider, style=inslider_style, layout=inslider_layout)

# %matplotlib ipympl

In [ ]:
acc = 250
sweep_speed = 50
t_acc, len_acc = cal_acc_time_length(acc, speed=sweep_speed)
print(f"加速时间: {t_acc:.2f}秒, 加速长度: {len_acc:.2f}mm")
config_path = "./config.toml"
timestamps_path = "./timestamps.txt"

offset_x = IntSlider(value=0, min=0, max=970, step=1, description="水平扫描偏移(mm)")
offset_y = IntSlider(value=0, min=0, max=1970, step=1, description="垂直扫描偏移(mm)")

fmc4030 = FMC4030()
with Braket(fmc4030) as braket:
    with braket.break_conrtol():
        time.sleep(2)
    braket.jog_x(0)
    braket.jog_y(0)
    braket.home_axis()


@widgets.interact(offset_x=offset_x, offset_y=offset_y)
def to_offset(offset_x, offset_y):
    with Braket(fmc4030) as braket:
        braket.jog_x(offset_x + len_acc)
        braket.jog_y(offset_y)

In [ ]:
sweep_dx = IntSlider(value=1, min=0.6, max=4, step=0.1, description="水平扫描间距(mm)")
sample_points = IntSlider(value=401, min=1, max=901, step=1, description="水平采样点数(mm)")  # 采样点数

sweep_lines = IntSlider(value=151, min=1, max=300, step=1, description="垂直扫描线数")
sweep_dy = IntSlider(value=2, min=1, max=86, step=1, description="垂直扫描间距(mm)")


@widgets.interact(sweep_dx=sweep_dx, sample_points=sample_points, sweep_lines=sweep_lines, sweep_dy=sweep_dy)
def set_config(sweep_dx, sample_points, sweep_lines, sweep_dy):
    framePeriodicity = 1000 * sweep_dx / sweep_speed  # 水平采样周期 ms
    sweep_length = sweep_dx * (sample_points - 1) + 2 * len_acc  # 水平扫描长度 mm
    print(f"framePriodicity: {framePeriodicity} ms")

    axis_info = [(sweep_length, sweep_speed, acc, acc), (sweep_dy, 150, 200, 200)]
    record_time = cal_all_time(axis_info, sweep_lines) + 10
    print(f"record_time:{record_time} s")

    num_frames = int(record_time * 1000 / framePeriodicity)
    print(f"num_frames:{num_frames}")

    mmw_cfg.mimo.frame.framePeriodicity = framePeriodicity
    mmw_cfg.mimo.frame.numLoops = 2
    mmw_cfg.mimo.frame.numFrames = num_frames

    mmw_cfg.bracket.profile.dx = sweep_dx
    mmw_cfg.bracket.profile.dy = sweep_dy
    mmw_cfg.bracket.profile.row = sweep_lines
    mmw_cfg.bracket.profile.col = sample_points
    mmw_cfg.bracket.profile.timestamps = "timestamps.txt"
    mmw_cfg.bracket.profile.offset_time = -0.920  # 手动偏移校准
    mmw_cfg.bracket.profile.record_time = record_time
    mmw_cfg.bracket.profile.next_line_reverse = True
    mmw_cfg.bracket.profile.pre_acc = True

In [ ]:
@widgets.interact_manual()
def sweep_next_line_reverse():
    now_date = datetime.datetime.now().strftime("%Y%m%d_%H%M")[2:]
    print(now_date)
    mmwave_dir = f"mmw_{now_date}"

    mmw_cfg.bracket.profile.run_time = now_date
    turn_toml(config_path, mmw_cfg.model_dump(exclude_none=True))

    mmwave = MMWaveCmd(config_file=config_path)

    record_time = mmw_cfg.bracket.profile.record_time
    framePeriodicity = mmw_cfg.mimo.frame.framePeriodicity
    sweep_length = sweep_dx.value * (sample_points.value - 1) + 2 * len_acc
    time_list = []
    jog_x_delay = 0
    with mmwave.record(mmwave_dir, record_time):
        st = time.time()
        with Braket(fmc4030) as braket:
            braket.jog_x(offset_x.value)

            for y_line in trange(sweep_lines.value):
                braket.jog_y(y_line * sweep_dy.value + offset_y.value)
                time.sleep(cal_last(time.time() - st + t_acc + jog_x_delay, framePeriodicity / 1000))
                start_time = time.time()
                if y_line % 2 == 0:
                    braket.jog_x(sweep_length + offset_x.value, speed=sweep_speed, acc=acc)
                else:
                    braket.jog_x(0 + offset_x.value, speed=sweep_speed, acc=acc)

                record_end = time.time() - st - t_acc + jog_x_delay
                record_start = start_time - st + t_acc + jog_x_delay

                tqdm.write(f"{(record_start, record_end)}")
                time_list.append((record_start, record_end))

            braket.jog_x(0)
            braket.jog_y(0)

        time_offset = mmwave.sync_time(st)
        time_list.append([time_offset, 0])
        np.savetxt(timestamps_path, np.array(time_list))

        wait_time = record_time - (time.time() - st)
        time.sleep(max(0, wait_time))

In [ ]:
def get_dev_files(dir_path: str):
    res = subprocess.run(["ssh", "root@192.168.33.180", f"ls /mnt/ssd/{dir_path}"], capture_output=True)
    res = res.stdout.decode()
    files = [i for i in res.split()]
    return files


@widgets.interact_manual()
def ls_file():
    now_date = mmw_cfg.bracket.profile.run_time
    mmwave_dir = f"mmw_{now_date}"

    files = get_dev_files(mmwave_dir)
    print(files)

In [ ]:
from pathlib import Path
from mmwave.util import load_config


@widgets.interact_manual()
def copy_file():
    mmw_cfg = load_config(config_path)
    now_date = mmw_cfg.bracket.profile.run_time
    mmwave_dir = f"mmw_{now_date}"

    # kill apps.out
    cmd = ["ssh", "root@192.168.33.180", "-C", "killall apps.out"]
    subprocess.run(cmd)

    files = get_dev_files(mmwave_dir)
    if config_path[2:] not in files:
        subprocess.run(["rsync", "-av", config_path, f"root@192.168.33.180:/mnt/ssd/{mmwave_dir}/"])
    if timestamps_path[2:] not in files:
        subprocess.run(["rsync", "-av", timestamps_path, f"root@192.168.33.180:/mnt/ssd/{mmwave_dir}/"])

    # change there!!!!!!!!
    tartgetdir_path = f"/media/ubuntu/tomin/{mmwave_dir}"
    Path(tartgetdir_path).mkdir(exist_ok=True)
    for f in tqdm(get_dev_files(mmwave_dir)):
        cmd = [
            "rsync",
            "-av",
            "--progress",
            "--whole-file",
            f"root@192.168.33.180:/mnt/ssd/{mmwave_dir}/{f}",
            f"{tartgetdir_path}/{f}",
        ]
        subprocess.run(cmd)